# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846911372487                   -0.70    4.8         
  2   -7.852330905673       -2.27       -1.53    1.0   22.7ms
  3   -7.852616196073       -3.54       -2.56    1.5   24.8ms
  4   -7.852645980541       -4.53       -2.90    2.2   30.3ms
  5   -7.852646506584       -6.28       -3.19    1.0   23.0ms
  6   -7.852646677558       -6.77       -4.07    1.0   23.0ms
  7   -7.852646686695       -8.04       -5.27    2.2   30.2ms
  8   -7.852646686727      -10.50       -5.55    1.8   26.6ms
  9   -7.852646686728      -12.20       -5.57    1.2   23.8ms
 10   -7.852646686730      -11.65       -6.78    1.0   23.3ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846821367489                   -0.70           4.8         
  2   -7.852527486481       -2.24       -1.63   0.80    2.0    251ms
  3   -7.852636740785       -3.96       -2.72   0.80    1.0   21.1ms
  4   -7.852646557270       -5.01       -3.27   0.80    2.2   28.3ms
  5   -7.852646678280       -6.92       -4.14   0.80    1.0   21.5ms
  6   -7.852646686345       -8.09       -4.83   0.80    1.5   24.6ms
  7   -7.852646686721       -9.42       -5.90   0.80    2.0   26.8ms
  8   -7.852646686730      -11.06       -6.67   0.80    2.2   28.6ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.364467e+01     2.949590e+00
 * time: 0.05624818801879883
     1     1.223375e+00     2.484281e+00
 * time: 0.27179813385009766
     2    -1.558582e+00     1.969897e+00
 * time: 0.29441118240356445
     3    -3.734994e+00     1.977191e+00
 * time: 0.3270900249481201
     4    -5.041772e+00     1.825586e+00
 * time: 0.36009907722473145
     5    -6.840234e+00     1.365782e+00
 * time: 0.3931729793548584
     6    -7.509285e+00     4.614332e-01
 * time: 0.42630910873413086
     7    -7.711706e+00     1.322901e-01
 * time: 0.44995903968811035
     8    -7.782526e+00     1.523861e-01
 * time: 0.4729330539703369
     9    -7.819182e+00     7.849215e-02
 * time: 0.4957890510559082
    10    -7.836291e+00     6.150202e-02
 * time: 0.5185849666595459
    11    -7.845641e+00     4.152645e-02
 * time: 0.5413579940795898
    12    -7.850329e+00     2.514169e-02
 * time: 0.564399003982544
    13    -7.851939e+00     1.233377e-02
 * time: 0.58717

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846832689320                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645896452                   -1.64         
  2   -7.852646686730       -6.10       -3.71    1.87s
  3   -7.852646686730      -13.25       -7.24    193ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.6054621608872974e-7
|ρ_newton - ρ_scfv| = 1.9907441795466904e-8
|ρ_newton - ρ_dm|   = 8.251363941160852e-10
